Librerias

In [ ]:
import os
import pandas as pd
from sqlalchemy import create_engine, String, inspect, MetaData, Table, Column
from sqlalchemy_utils import database_exists, create_database

Funcion para leer las credenciales (Se necesita el archivo Credenciales.txt en la ubicacion de la carpeta)

In [26]:
credentials = {}
with open("credenciales.txt", "r") as file:
    for line in file:
        key, value = line.strip().split("=")
        credentials[key] = value

user = credentials["PG_USER"]
password = credentials["PG_PASSWORD"]
host = credentials["PG_HOST"]
port = credentials["PG_PORT"]
database = credentials["PG_DATABASE"]

Se crea la URL y el motor de conexion (sqlalchemy)

In [28]:
url = f"postgresql://{user}:{password}@{host}:{port}/{database}"
engine = create_engine(url)

Revisar si exite ya la base de datos o no

In [29]:
if not database_exists(engine.url):
    create_database(engine.url)
    print(f"Base de datos '{database}' creada.")
else:
    print(f"Conectando a la base de datos existente '{database}'.")

Conectando a la base de datos existente 'postgres'.


Leer la base de datos en el achivo CSV (Revisar la ruta)

In [ ]:
csv_file = "./data/survey_results_public.csv"
print("Leyendo el archivo CSV...")
df = pd.read_csv(csv_file)
print(f"Archivo CSV leido: {df.shape[0]} filas, {df.shape[1]} columnas.")

Leyendo el archivo CSV...


C:\Users\Nicolas Cuaran\AppData\Local\Temp\ipykernel_18544\1604996450.py:3: DtypeWarning: Columns (8,12,13,14,15,16,50,51,52,53,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_file)


Archivo CSV leído: 98855 filas, 129 columnas.


Logica para inferir tipos de datos para SQL (sqlalchemy)

In [33]:
def infer_sqlalchemy_type(dtype, column_name):
    # Determinar la longitud máxima necesaria para cada columna
    max_length = df[column_name].str.len().max() if pd.api.types.is_string_dtype(dtype) else 500
    if max_length > 2000:  # Límite práctico para evitar problemas extremos
        max_length = 2000
    return String(max_length)

 Verifica la tabla si no exite y sube los datos

In [36]:
table_name = "surevey_results"
if not inspect(engine).has_table(table_name):
    metadata = MetaData()
    columns = [Column(name,
                      infer_sqlalchemy_type(dtype, name),
                      primary_key=(name == 'eventid')) \
                          for name, dtype in df.dtypes.items()]
    
    table = Table(table_name, metadata, *columns)
    table.create(engine)
    print(f"Tabla '{table_name}' creada.")
    
    # Insertar datos del DataFrame
    df.to_sql(table_name, con=engine, if_exists='append', index=False)
    print(f"Datos insertados en la tabla '{table_name}'.")
else:
    print(f"La tabla '{table_name}' ya existe.")

Tabla 'surevey_results' creada.
Datos insertados en la tabla 'surevey_results'.
